Initialization.
To be run once every start of sessions.

In [ ]:
!pip install simupy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 416.5 kB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pickle
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import matplotlib.pyplot as plt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Code some additional information from the input-output tables on one year before generalization.
 - Loading necessary items for the simulation. Redefining some is necessary to access deeper information.


In [ ]:
# Define the function to load variables
from google.colab import drive
drive.mount('/content/drive')

import pickle
import pandas as pd

# Define the file path in Google Drive
file_path = '/content/drive/MyDrive/Yannis/all_variables.pkl'

def load_variables(year):
    load_path = f'/content/drive/MyDrive/Yannis/variables_{year}.pkl'
    with open(load_path, 'rb') as file:
        variables = pickle.load(file)
    return variables

# Load Data and Initialize Variables
year_to_load = 2016
loaded_variables = load_variables(year_to_load)

Eora_pp_FD = loaded_variables['Eora_pp_FD']
labels_FD = loaded_variables['labels_FD']
Eora_pp_Q = loaded_variables['Eora_pp_Q']
labels_Q = loaded_variables['labels_Q']
Eora_pp_QY = loaded_variables['Eora_pp_QY']
Eora_pp_T = loaded_variables['Eora_pp_T']
labels_T = loaded_variables['labels_T']
Eora_pp_VA = loaded_variables['Eora_pp_VA']
labels_VA = loaded_variables['labels_VA']
Eora_pp_T_matrix = loaded_variables['Eora_pp_T_matrix']
Eora_pp_Q_emissions_matrix = loaded_variables['Eora_pp_Q_emissions_matrix']
xout = loaded_variables['xout']
A = loaded_variables['A']
f = loaded_variables['f']
I = loaded_variables['I']
L = loaded_variables['L']
sector_labels = loaded_variables['sector_labels']

# Function to move 'label' column to the first position
def move_label_to_first_column(df):
    label_column = df.pop('label')
    df.insert(0, 'label', label_column)
    return df

# Add labels to dataframe and move 'label' column to the first position
for df, col_index, eora in zip([labels_FD, labels_Q, labels_T, labels_VA], [3, 1, 3, 1], [Eora_pp_FD, Eora_pp_Q, Eora_pp_T, Eora_pp_VA]):
    eora['label'] = df.iloc[:, col_index]
    eora = move_label_to_first_column(eora)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- Establish data for solar panels emissions and solar panels emissions

In [ ]:
Eora_pp_Q.head(10)

label        0         1        2  \
0                             Natural Gas  14.5665  0.424861  6.35112   
1                                    Coal  11.8107  0.344482  5.14956   
2                               Petroleum   0.0000  0.000000  0.00000   
3                     Nuclear Electricity   0.0000  0.000000  0.00000   
4               Hydroelectric Electricity   0.0000  0.000000  0.00000   
5                  Geothermal Electricity   0.0000  0.000000  0.00000   
6                        Wind Electricity   0.0000  0.000000  0.00000   
7        Solar, Tide and Wave Electricity   0.0000  0.000000  0.00000   
8           Biomass and Waste Electricity   0.0000  0.000000  0.00000   
9  Public electricity and heat production   1.5750  1.575000  1.57500   

         3        4        5        6        7        8  ...      4905  \
0  32.6228  5.95376  15.3731  42.8622  17.4204  44.2079  ...    0.0000   
1  26.4509  4.82737  12.4647  34.7532  14.1247  35.8443  ...  683.7270   
2   0.0000  0.00000   0.0000   0.0000   0.0000   0.0000  ...    7.7684   
3   0.0000  0.00000   0.0000   0.0000   0.0000   0.0000  ...    0.0000   
4   0.0000  0.00000   0.0000   0.0000   0.0000   0.0000  ...    0.0000   
5   0.0000  0.00000   0.0000   0.0000   0.0000   0.0000  ...    0.0000   
6   0.0000  0.00000   0.0000   0.0000   0.0000   0.0000  ...    0.0000   
7   0.0000  0.00000   0.0000   0.0000   0.0000   0.0000  ...    0.0000   
8   0.0000  0.00000   0.0000   0.0000   0.0000   0.0000  ...    7.7714   
9   1.5750  1.57500   1.5750   1.5750   1.5750   1.5750  ...    0.1000   

         4906       4907      4908       4909       4910       4911  \
0      0.0000    0.00000     0.000     0.0000     0.0000    0.00000   
1   1696.1200  594.74900  1808.420  1395.8900  1189.2000  159.37400   
2  17262.7000    6.75744    20.547    15.8599    13.5115    1.81078   
3      0.0000    0.00000     0.000     0.0000     0.0000    0.00000   
4      0.0000    0.00000     0.000     0.0000     0.0000    0.00000   
5      0.0000    0.00000     0.000     0.0000     0.0000    0.00000   
6      0.0000    0.00000     0.000     0.0000     0.0000    0.00000   
7      0.0000    0.00000     0.000     0.0000     0.0000    0.00000   
8     13.0921    6.76005    20.555    15.8660    13.5167    1.81148   
9      0.1000    0.10000     0.100     0.1000     0.1000    0.10000   

         4912  4913  4914  
0     0.00000   0.0   0.0  
1     0.00000   0.0   0.0  
2  2723.33000   0.0   0.0  
3     0.00000   0.0   0.0  
4     0.00000   0.0   0.0  
5     0.00000   0.0   0.0  
6     0.00000   0.0   0.0  
7     0.00000   0.0   0.0  
8     1.91107   0.0   0.0  
9     0.10000   0.0   0.0  

[10 rows x 4916 columns]

In [ ]:
# Extract energy-related labels from T
energy_condition = Eora_pp_T['label'].str.contains('Electricity, Gas and Water', case=False, na=False)
energy_Eora_pp_T = Eora_pp_T.loc[energy_condition]
solar_panels_production = energy_Eora_pp_T #If we want to use it as a proxy for the first tries

# Extract solar-related emissions from Q
solar_panels_condition = Eora_pp_Q['label'].str.contains('Solar, Tide and Wave Electricity', case=False, na=False)
solar_panels_emissions = Eora_pp_Q.loc[solar_panels_condition]


# Combine the production and emissions data
solar_panels_data = pd.concat([solar_panels_production, solar_panels_emissions.iloc[[0]]], ignore_index=True)
print(solar_panels_data.head(190))

                                label           0         1           2  \
0          Electricity, Gas and Water  7676.10000  28.14770  8088.37000   
1          Electricity, Gas and Water    -2.61973  -1.80189     4.72827   
2          Electricity, Gas and Water    -2.56368  -1.96346     1.86361   
3          Electricity, Gas and Water    -1.81884  -1.17345     2.03870   
4          Electricity, Gas and Water    -4.12896  -3.02453     3.17007   
..                                ...         ...       ...         ...   
185        Electricity, Gas and Water     1.71845   2.23021     5.85164   
186        Electricity, Gas and Water    -2.51296  -1.88869     1.29926   
187        Electricity, Gas and Water    -4.43778  -3.17718     5.74336   
188        Electricity, Gas and Water     1.03271   1.34573     2.93955   
189  Solar, Tide and Wave Electricity     0.00000   0.00000     0.00000   

               3  ...       4910       4911       4912       4913       4914  
0    12415.60000  ..

Use the previously defined variables to build a simple systems dynamics model, by defining a function that :
- Selects the Country to be analyzed
- Selects the sector to be analyzed
- Selects the variables needed
- Describes the initial state
- Describes the time span
- Describes the feedback loops
- Describes the interlinkages

In [ ]:
# Initial state and time span for simulation
initial_state = np.array([initial_emissions, initial_energy_production, initial_labor_cost, initial_taxes, initial_subsidies])
time_span = np.linspace(0, 50, 500)  # Simulate for 50 years

# Simulate the system
results = system.simulate(initial_state, time_span)


In [ ]:
from simupy.systems.symbolic import DynamicalSystem
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

# Define initial conditions and parameters
initial_emissions = Eora_pp_Q.sum(axis=1).sum()  # Sum emissions over all sectors and regions
energy_rows = labels_T[labels_T[3].str.contains('Electricity, Gas and Water', case=False, na=False)] # Step 1: Identify rows in labels_T where column 3 contains 'Electricity, Gas and Water'
energy_indices = energy_rows.index # Step 2: Use the indices of these rows to filter Eora_pp_T
energy_data = Eora_pp_T.loc[energy_indices]
initial_energy_production = energy_data.sum(axis=1)  ## Step 3: Sum the values of the filtered rows in Eora_pp_T
Worldwide_energy_production = energy_data.sum(axis=1).sum(axis=0)
initial_solar_labor_cost = Eora_pp_VA[:, labels_VA.index('Compensation of employees D.1')].sum()  # Total labor cost
initial_energy_labor_cost = Eora_pp_VA[:, labels_VA.index('Compensation of employees D.1')].sum()  # Total labor cost
initial_taxes = Eora_pp_VA[:, labels_VA.index('Taxes on production D.29')].sum()  # Total taxes
initial_subsidies = Eora_pp_VA[:, labels_VA.index('Subsidies on production D.39')].sum()  # Total subsidies

# Define symbols for the state variables and parameters
t = sp.Symbol('t')
emissions = sp.Symbol('emissions')
energy_production = sp.Symbol('energy_production')
labor_cost = sp.Symbol('labor_cost')
taxes = sp.Symbol('taxes')
subsidies = sp.Symbol('subsidies')
solar_integration_rate = sp.Symbol('solar_integration_rate')
emission_reduction_factor = sp.Symbol('emission_reduction_factor')

# Define the model equations
d_emissions_dt = -emission_reduction_factor * solar_integration_rate * energy_production
d_energy_production_dt = solar_integration_rate * energy_production
d_labor_cost_dt = -solar_integration_rate * labor_cost  # Assume labor cost decreases with solar integration
d_taxes_dt = solar_integration_rate * taxes  # Assume taxes increase with solar integration
d_subsidies_dt = -solar_integration_rate * subsidies  # Assume subsidies decrease as solar becomes more integrated

state = sp.Matrix([emissions, energy_production, labor_cost, taxes, subsidies])
state_derivative = sp.Matrix([d_emissions_dt, d_energy_production_dt, d_labor_cost_dt, d_taxes_dt, d_subsidies_dt])
params = {
    solar_integration_rate: 0.05,  # Example rate of solar panel integration
    emission_reduction_factor: 0.3  # Example reduction factor for emissions
}

# Create the dynamical system
system = DynamicalSystem(state, t, state_derivative, params=params)

# Initial state and time span for simulation
initial_state = np.array([initial_emissions, initial_energy_production, initial_labor_cost, initial_taxes, initial_subsidies])
time_span = np.linspace(0, 50, 500)  # Simulate for 50 years

# Simulate the system
results = system.simulate(initial_state, time_span)

# Extract results
emissions_results = results[:, 0]
energy_production_results = results[:, 1]
labor_cost_results = results[:, 2]
taxes_results = results[:, 3]
subsidies_results = results[:, 4]

# Plot the results
plt.figure(figsize=(12, 8))
plt.plot(time_span, emissions_results, label='Emissions')
plt.plot(time_span, energy_production_results, label='Energy Production')
plt.plot(time_span, labor_cost_results, label='Labor Cost')
plt.plot(time_span, taxes_results, label='Taxes')
plt.plot(time_span, subsidies_results, label='Subsidies')
plt.xlabel('Time (years)')
plt.ylabel('Quantity')
plt.legend()
plt.title('Impact of Solar Panel Integration on Emissions, Energy Production, and Economic Variables')
plt.grid(True)
plt.show()


TypeError: 'RangeIndex' object is not callable